# Assignment 8: DT

<ol>
    <li><strong>Apply Decision Tree Classifier(DecisionTreeClassifier) on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical, numerical features +  preprocessed_eassay (TFIDF)</li>
            <li><font color='red'>Set 2</font>: categorical, numerical features +  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (best `depth` in range [1, 5, 10, 50], and the best `min_samples_split` in range [5, 10, 100, 500])</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation(use gridsearch cv or randomsearch cv)/simple cross validation data(you can write your own for loops refer sample solution)</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>min_sample_split</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
    <li>Once after you plot the confusion matrix with the test data, get all the `false positive data points`
        <ul>
            <li> Plot the WordCloud(https://www.geeksforgeeks.org/generating-word-cloud-python/) with the words of essay text of these `false positive data points`</li>
            <li> Plot the box plot with the `price` of these `false positive data points`</li>
            <li> Plot the pdf with the `teacher_number_of_previously_posted_projects` of these `false positive data points`</li>
        </ul>
        </ul>
    </li>
   <li><b>Task 2: </b>For this task consider set-1 features. Select all the features which are having non-zero feature importance.You can get the feature importance using  'feature_importances_` 
   (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), discard the all other remaining features and then apply any of the model of you choice i.e. (Dession tree, Logistic Regression, Linear SVM), you need to do hyperparameter tuning corresponding to the model you selected and procedure in step 2 and step 3<br>
  Note: when you want to find the feature importance make sure you don't use max_depth parameter keep it None.
  </li>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

<h1>1. Decision Tree </h1>

## 1.1 Loading Data

In [1]:
import pandas
data = pandas.read_csv('preprocessed_data.csv')
print(data.shape)
data.head()

(109248, 9)


,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74


<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [2]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label
## Import required library 
from sklearn.model_selection import train_test_split

## create dependent and independent varaible
y_df = data['project_is_approved'].values
X_df = data.drop('project_is_approved',axis=1)

## create train,test split 
X_tr,X_test,y_tr,y_test = train_test_split(X_df, y_df, test_size=0.33, random_state=42,stratify=y_df)
print(X_tr.shape, y_tr.shape)
print(X_test.shape, y_test.shape)

(73196, 8) (73196,)
(36052, 8) (36052,)


<h2>1.3 Make Data Model Ready: encoding eassay, and project_title</h2>

In [3]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tqdm import tqdm
import pickle

In [5]:
def TFIDF_essays_sub(X_train, y_train, X_test, y_test, col_name = 'essay'):
    
    vectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,4),max_features=500)
    vectorizer.fit(X_train[col_name].values) # fit has to happen only on train data

    # we use the fitted CountVectorizer to convert the text to vector
    X_train_ohe =   vectorizer.transform(X_train[col_name].values)
    X_test_ohe = vectorizer.transform(X_test[col_name].values)

    print("After vectorizations {}".format(col_name))
    print(X_train_ohe.shape, y_train.shape)
    print(X_test_ohe.shape, y_test.shape)
    #print(vectorizer.get_feature_names())
    print("="*100)
    
    return X_train_ohe, X_test_ohe, vectorizer.get_feature_names()

In [6]:
%%time
X_tr_essay_tfidf, X_test_essay_tfidf,TFIDF_feature_name = TFIDF_essays_sub(X_tr, y_tr, X_test, y_test, col_name = 'essay')

After vectorizations essay
(73196, 500) (73196,)
(36052, 500) (36052,)
Wall time: 7min


In [7]:
%%time
# Using Pretrained Model - TFIDF weighted W2V
# Train Essays
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(X_tr["essay"])
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_vec.get_feature_names(), list(tfidf_vec.idf_)))
tfidf_words = set(tfidf_vec.get_feature_names())

Wall time: 24.8 s


In [8]:
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [9]:
%%time
# average Word2Vec
# compute average word2vec for each review.
X_tr_essay_wghtd_w2vec = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_tr["essay"]): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    X_tr_essay_wghtd_w2vec.append(vector)

print(len(X_tr_essay_wghtd_w2vec))

100%|██████████| 73196/73196 [07:53<00:00, 154.73it/s]

73196
Wall time: 7min 53s


In [11]:
%%time
# average Word2Vec
# compute average word2vec for each review.
X_test_essay_wghtd_w2vec = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test["essay"]): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    X_test_essay_wghtd_w2vec.append(vector)

print(len(X_test_essay_wghtd_w2vec))

100%|██████████| 36052/36052 [02:21<00:00, 254.28it/s]

36052
Wall time: 2min 21s


<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [10]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [12]:
def one_hot_encoding(X_train, y_train, X_test, y_test,col_name):
    vectorizer = CountVectorizer()
    vectorizer.fit(X_train[col_name].values) # fit has to happen only on train data

    # we use the fitted CountVectorizer to convert the text to vector
    X_tr_ohe = vectorizer.transform(X_train[col_name].values)
    X_test_ohe = vectorizer.transform(X_test[col_name].values)

    print("After vectorizations {}".format(col_name))
    print(X_tr_ohe.shape, y_train.shape)
    print(X_test_ohe.shape, y_test.shape)
    print(vectorizer.get_feature_names())
    print("="*100)
    
    return X_tr_ohe, X_test_ohe, vectorizer.get_feature_names()

In [13]:
X_tr_state_ohe, X_test_state_ohe, state_feature_name = one_hot_encoding(X_tr, y_tr, X_test, y_test,'school_state')

After vectorizations school_state
(73196, 51) (73196,)
(36052, 51) (36052,)
['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']


In [14]:
X_tr_pgc_ohe, X_test_pgc_ohe, pgc_feature_name = one_hot_encoding(X_tr, y_tr, X_test, y_test,'project_grade_category')

After vectorizations project_grade_category
(73196, 4) (73196,)
(36052, 4) (36052,)
['grades_3_5', 'grades_6_8', 'grades_9_12', 'grades_prek_2']


In [15]:
X_tr_tpr_ohe, X_test_tpr_ohe, tp_feature_name = one_hot_encoding(X_tr, y_tr, X_test, y_test,'teacher_prefix')
print(len(tp_feature_name))

After vectorizations teacher_prefix
(73196, 5) (73196,)
(36052, 5) (36052,)
['dr', 'mr', 'mrs', 'ms', 'teacher']
5


In [16]:
X_tr_cln_catg_ohe, X_test_cln_catg_ohe, cln_cat_feature_name = one_hot_encoding(X_tr, y_tr, X_test, y_test,'clean_categories') 

After vectorizations clean_categories
(73196, 9) (73196,)
(36052, 9) (36052,)
['appliedlearning', 'care_hunger', 'health_sports', 'history_civics', 'literacy_language', 'math_science', 'music_arts', 'specialneeds', 'warmth']


In [17]:
X_tr_cln_sub_catg_ohe, X_test_cln_sub_catg_ohe, cln_sub_catg_feature_name = one_hot_encoding(X_tr, y_tr, X_test, y_test,'clean_subcategories')

After vectorizations clean_subcategories
(73196, 30) (73196,)
(36052, 30) (36052,)
['appliedsciences', 'care_hunger', 'charactereducation', 'civics_government', 'college_careerprep', 'communityservice', 'earlydevelopment', 'economics', 'environmentalscience', 'esl', 'extracurricular', 'financialliteracy', 'foreignlanguages', 'gym_fitness', 'health_lifescience', 'health_wellness', 'history_geography', 'literacy', 'literature_writing', 'mathematics', 'music', 'nutritioneducation', 'other', 'parentinvolvement', 'performingarts', 'socialsciences', 'specialneeds', 'teamsports', 'visualarts', 'warmth']


In [18]:
### Normalize your data 
def norm_data(X_tr,y_tr, X_test, y_test, col_name = 'price'):
    
    from sklearn.preprocessing import Normalizer
    normalizer = Normalizer()

    normalizer.fit(X_tr['price'].values.reshape(1,-1))

    X_tr_norm = normalizer.transform(X_tr['price'].values.reshape(-1,1))
    X_test_norm = normalizer.transform(X_test['price'].values.reshape(-1,1))


    print("After vectorizations")
    print(X_tr_norm.shape, y_tr.shape)
    print(X_test_norm.shape, y_test.shape)
    print("="*100)
    return X_tr_norm,X_test_norm

In [19]:
X_tr_price_norm, X_test_price_norm = norm_data(X_tr, y_tr, X_test, y_test,'price')

After vectorizations
(73196, 1) (73196,)
(36052, 1) (36052,)


In [20]:
X_tr_nopp_norm, X_test_nopp_norm = norm_data(X_tr, y_tr, X_test, y_test,'teacher_number_of_previously_posted_project')

After vectorizations
(73196, 1) (73196,)
(36052, 1) (36052,)


In [21]:
print(X_tr_essay_tfidf.shape)
print(X_tr_state_ohe.shape)
print(X_tr_pgc_ohe.shape)
print(X_tr_tpr_ohe.shape)
print(X_tr_cln_catg_ohe.shape)
print(X_tr_cln_sub_catg_ohe.shape)
print(X_tr_price_norm.shape)
print(X_tr_nopp_norm.shape)

(73196, 500)
(73196, 51)
(73196, 4)
(73196, 5)
(73196, 9)
(73196, 30)
(73196, 1)
(73196, 1)


In [22]:
## prepare your data set to train your model
from scipy.sparse import hstack
X_tr_tfidf_csr = hstack((X_tr_essay_tfidf,X_tr_state_ohe, X_tr_pgc_ohe, X_tr_tpr_ohe, X_tr_cln_catg_ohe,X_tr_cln_sub_catg_ohe,X_tr_nopp_norm,X_tr_price_norm)).tocsr()
X_test_tfidf_csr = hstack((X_test_essay_tfidf,X_test_state_ohe, X_test_pgc_ohe, X_test_tpr_ohe, X_test_cln_catg_ohe,X_test_cln_sub_catg_ohe,X_test_nopp_norm,X_test_price_norm)).tocsr()

print("Final Data matrix")
print(X_tr_tfidf_csr.shape, y_tr.shape)
print(X_test_tfidf_csr.shape, y_test.shape)
print("="*100)

Final Data matrix
(73196, 601) (73196,)
(36052, 601) (36052,)


In [23]:
## prepare your data set to train your model
from scipy.sparse import hstack
X_tr_wghtd_w2v_csr = hstack((X_tr_essay_wghtd_w2vec,X_tr_state_ohe, X_tr_pgc_ohe, X_tr_tpr_ohe, X_tr_cln_catg_ohe,X_tr_cln_sub_catg_ohe,X_tr_nopp_norm,X_tr_price_norm)).tocsr()
X_test_wghtd_w2v_csr = hstack((X_test_essay_wghtd_w2vec,X_test_state_ohe, X_test_pgc_ohe, X_test_tpr_ohe, X_test_cln_catg_ohe,X_test_cln_sub_catg_ohe,X_test_nopp_norm,X_test_price_norm)).tocsr()

print("Final Data matrix")
print(X_tr_wghtd_w2v_csr.shape, y_tr.shape)
print(X_test_wghtd_w2v_csr.shape, y_test.shape)
print("="*100)

Final Data matrix
(73196, 401) (73196,)
(36052, 401) (36052,)


<h2>1.5 Appling  Decision Tree on different kind of featurization as mentioned in the instructions</h2>

<br>Apply  Decision Tree on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

In [28]:
### Import required library
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
## Perform grid search cv to tune parameter for max_dept and min_sample_split

hyper_parameter = {'max_depth': [1, 5, 10, 50, 100, 500, 100], \
                  'min_samples_split': [5, 10, 100, 500]}

dtclf = DecisionTreeClassifier(class_weight='balanced') 
clf = GridSearchCV(dtclf, hyper_parameter, cv=10, scoring='roc_auc', return_train_score=True, n_jobs=-1)
clf.fit(X_tr_tfidf_csr,y_tr)

In [ ]:
train_auc= clf.cv_results_['mean_train_score']
train_auc_std = clf.cv_results_['std_train_score']
test_auc = clf.cv_results_['mean_test_score'] 
test_auc_std = clf.cv_results_['std_test_score']

#Output of GridSearchCV
print('Best score: ',clf.best_score_)
print('Best Hyper parameters: ',clf.best_params_)
print('='*75)
print('Train AUC scores')
print(clf.cv_results_['mean_train_score'])
print('CV AUC scores')
print(clf.cv_results_['mean_test_score'])

<h2>1.6 Getting top features using `feature_importances_`</h2>

In [ ]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h1>2. Summary</h1>